In [71]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,RepeatedStratifiedKFold,cross_val_score
from numpy import mean,std
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score,precision_score,recall_score
import warnings 
warnings.filterwarnings('ignore')
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense

In [98]:
data = pd.read_csv('../Datasets/oversampled_data.csv')
del data['Unnamed: 0']
data.head()

,account_length,voice_mail_plan,voice_mail_messages,day_mins,evening_mins,night_mins,international_mins,customer_service_calls,international_plan,day_calls,day_charge,evening_calls,evening_charge,night_calls,night_charge,international_calls,international_charge,total_charge,churn
0,128,1,25,265.1,197.4,244.7,10.0,1,0,110,45.07,99,16.78,91,11.01,3,2.70,75.56,0
1,107,1,26,161.6,195.5,254.4,13.7,1,0,123,27.47,103,16.62,103,11.45,3,3.70,59.24,0
2,137,0,0,243.4,121.2,162.6,12.2,0,0,114,41.38,110,10.30,104,7.32,5,3.29,62.29,0
3,84,0,0,299.4,61.9,196.9,6.6,2,1,71,50.90,88,5.26,89,8.86,7,1.78,66.80,0
4,75,0,0,166.7,148.3,186.9,10.1,3,1,113,28.34,122,12.61,121,8.41,3,2.73,52.09,0


### Scaling an dseperating X and y

In [94]:
std_scr = StandardScaler()
X = data.drop(labels='churn',axis=1)
cols = X.columns
X = pd.DataFrame(std_scr.fit_transform(X))
X.columns = cols
y = data[['churn']]
X.shape,y.shape

((3333, 18), (3333, 1))

### Train Test Split

In [95]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,shuffle=True,random_state=12,stratify=y)

### Dict to store performance metrics

In [96]:
my_dict = {'model':[], 'accuracy': [],'precision':[],'recall':[]}

### Method to predit and calculate model performance

In [97]:
def populate_matrix(fittedModel):
    y_pred = fittedModel.predict(x_test)
    accuracy = round(accuracy_score(y_test,y_pred)*100,2)
    precision = round(precision_score(y_test,y_pred)*100,2)
    recall = round(recall_score(y_test,y_pred)*100,2)
    model = str(fittedModel)
    my_dict["model"].append(model)
    my_dict["accuracy"].append(accuracy)
    my_dict["precision"].append(precision)
    my_dict["recall"].append(recall)


## Logistic Regression

In [77]:
# evaluate model
logistic = LogisticRegression()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=logistic,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))


Accuracy: 0.776 (0.016)


### Model Traning

In [78]:
logistic.fit(x_train,y_train)
populate_matrix(logistic)
print(my_dict)

{'model': ['LogisticRegression()'], 'accuracy': [76.75], 'precision': [75.63], 'recall': [78.95]}


## DecisionTree

In [79]:
dtree = DecisionTreeClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=dtree,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.983 (0.005)


### Model Training

In [80]:
dtree.fit(x_train,y_train)
populate_matrix(dtree)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()'], 'accuracy': [76.75, 98.42], 'precision': [75.63, 97.1], 'recall': [78.95, 99.82]}


## Random Forest

In [81]:
rf = RandomForestClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=rf,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.999 (0.001)


### Model Traning

In [82]:
rf.fit(x_train,y_train)
populate_matrix(rf)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()'], 'accuracy': [76.75, 98.42, 99.91], 'precision': [75.63, 97.1, 100.0], 'recall': [78.95, 99.82, 99.82]}


## Gradient Boosting

In [83]:
gboost = GradientBoostingClassifier()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=gboost,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.940 (0.008)


### Model Training

In [84]:
gboost.fit(x_train,y_train)
populate_matrix(gboost)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()'], 'accuracy': [76.75, 98.42, 99.91, 94.12], 'precision': [75.63, 97.1, 100.0, 99.8], 'recall': [78.95, 99.82, 99.82, 88.42]}


## SVM

In [85]:
svm = SVC()
# evaluate model
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(estimator=svm,X=X,y=y, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))

Accuracy: 0.911 (0.011)


### Model Training

In [86]:
svm.fit(x_train,y_train)
populate_matrix(svm)
print(my_dict)

{'model': ['LogisticRegression()', 'DecisionTreeClassifier()', 'RandomForestClassifier()', 'GradientBoostingClassifier()', 'SVC()'], 'accuracy': [76.75, 98.42, 99.91, 94.12, 89.3], 'precision': [75.63, 97.1, 100.0, 99.8, 89.44], 'recall': [78.95, 99.82, 99.82, 88.42, 89.12]}


## ANN

In [87]:
ann_model = Sequential()
ann_model.add(Dense(20,input_dim=x_train.shape[1],activation='relu'))
ann_model.add(Dense(10,activation='relu'))
ann_model.add(Dense(5,activation='relu'))
ann_model.add(Dense(1,activation='sigmoid'))


In [88]:
ann_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


### Model Training¶

In [89]:
ann_model.fit(x_train,y_train,batch_size=100,epochs=200,validation_data=(x_test,y_test),shuffle=True)



Epoch 1/200
46/46 [==============================] - 0s 3ms/step - loss: 0.6752 - accuracy: 0.5884 - val_loss: 0.6156 - val_accuracy: 0.6825
Epoch 2/200
46/46 [==============================] - 0s 1ms/step - loss: 0.5728 - accuracy: 0.7123 - val_loss: 0.5287 - val_accuracy: 0.7526
Epoch 3/200
46/46 [==============================] - 0s 2ms/step - loss: 0.4948 - accuracy: 0.7728 - val_loss: 0.4620 - val_accuracy: 0.7904
Epoch 4/200
46/46 [==============================] - 0s 2ms/step - loss: 0.4378 - accuracy: 0.8151 - val_loss: 0.4222 - val_accuracy: 0.8140
Epoch 5/200
46/46 [==============================] - 0s 1ms/step - loss: 0.4026 - accuracy: 0.8364 - val_loss: 0.3971 - val_accuracy: 0.8325
Epoch 6/200
46/46 [==============================] - 0s 1ms/step - loss: 0.3803 - accuracy: 0.8471 - val_loss: 0.3845 - val_accuracy: 0.8509
Epoch 7/200
46/46 [==============================] - 0s 2ms/step - loss: 0.3664 - accuracy: 0.8550 - val_loss: 0.3744 - val_accuracy: 0.8535
Epoch 8/200
4

Epoch 59/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1773 - accuracy: 0.9344 - val_loss: 0.2604 - val_accuracy: 0.9114
Epoch 60/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1782 - accuracy: 0.9351 - val_loss: 0.2511 - val_accuracy: 0.9088
Epoch 61/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1741 - accuracy: 0.9364 - val_loss: 0.2488 - val_accuracy: 0.9140
Epoch 62/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1722 - accuracy: 0.9342 - val_loss: 0.2591 - val_accuracy: 0.9096
Epoch 63/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1707 - accuracy: 0.9384 - val_loss: 0.2511 - val_accuracy: 0.9149
Epoch 64/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1687 - accuracy: 0.9364 - val_loss: 0.2529 - val_accuracy: 0.9175
Epoch 65/200
46/46 [==============================] - 0s 1ms/step - loss: 0.1676 - accuracy: 0.9375 - val_loss: 0.2502 - val_accuracy: 0.9132
Epoch 

46/46 [==============================] - 0s 1ms/step - loss: 0.0832 - accuracy: 0.9741 - val_loss: 0.2814 - val_accuracy: 0.9333
Epoch 174/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0819 - accuracy: 0.9746 - val_loss: 0.2945 - val_accuracy: 0.9316
Epoch 175/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0809 - accuracy: 0.9748 - val_loss: 0.2770 - val_accuracy: 0.9316
Epoch 176/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0807 - accuracy: 0.9754 - val_loss: 0.2713 - val_accuracy: 0.9316
Epoch 177/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0824 - accuracy: 0.9741 - val_loss: 0.2836 - val_accuracy: 0.9316
Epoch 178/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0798 - accuracy: 0.9763 - val_loss: 0.2963 - val_accuracy: 0.9351
Epoch 179/200
46/46 [==============================] - 0s 1ms/step - loss: 0.0800 - accuracy: 0.9757 - val_loss: 0.2863 - val_accuracy: 0.9298
Epoch 180/200

## convertiing probabilities to labels

In [90]:
y_pred = ann_model.predict(x_test)
for i in range(0,len(y_pred)):
    if y_pred[i]>=.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0

In [91]:
my_dict["model"].append('ANN')
my_dict["accuracy"].append(round(accuracy_score(y_test,y_pred)*100,2))
my_dict["precision"].append(round(precision_score(y_test,y_pred)*100,2))
my_dict["recall"].append(round(recall_score(y_test,y_pred)*100,2))

## Final performance matrix of all the models

In [92]:
pd.DataFrame(my_dict)

,model,accuracy,precision,recall
0,LogisticRegression(),76.75,75.63,78.95
1,DecisionTreeClassifier(),98.42,97.10,99.82
2,RandomForestClassifier(),99.91,100.00,99.82
3,GradientBoostingClassifier(),94.12,99.80,88.42
4,SVC(),89.30,89.44,89.12
5,ANN,93.60,90.54,97.37
